In [1]:
import numpy as np

import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt

%matplotlib inline

path = 'D:/CS/ML/DataFountain/unicom/'

train = pd.read_csv('%s%s' %(path, 'train.csv'))#, low_memory=False

test = pd.read_csv('%s%s' %(path, 'test.csv'))

F:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4,5,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
F:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
from numpy.core.umath_tests import inner1d
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.stats import skew
from sklearn.preprocessing import RobustScaler

from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVC
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor

F:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  """Entry point for launching an IPython kernel.


In [3]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier

In [4]:
train['2_total_fee'] = train['2_total_fee'].replace('\\N', 0)
train['3_total_fee'] = train['3_total_fee'].replace('\\N', 0)
train['age'] = train['age'].replace('\\N', 0)
train['gender'] = train['gender'].replace('\\N', 0)

test['2_total_fee'] = test['2_total_fee'].replace('\\N', 0)
test['3_total_fee'] = test['3_total_fee'].replace('\\N', 0)
test['age'] = test['age'].replace('\\N', 0)
test['gender'] = test['gender'].replace('\\N', 0)

In [5]:
train['gender'] = train['gender'].apply(lambda x : int(x))
test['gender'] = test['gender'].apply(lambda x : int(x))

train['age'] = train['age'].apply(lambda x : int(x))
test['age'] = test['age'].apply(lambda x : int(x))

train['2_total_fee'] = train['2_total_fee'].apply(lambda x : float(x))
test['2_total_fee'] = test['2_total_fee'].apply(lambda x : float(x))

train['3_total_fee'] = train['3_total_fee'].apply(lambda x : float(x))
test['3_total_fee'] = test['3_total_fee'].apply(lambda x : float(x))

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612652 entries, 0 to 612651
Data columns (total 27 columns):
service_type              612652 non-null int64
is_mix_service            612652 non-null int64
online_time               612652 non-null int64
1_total_fee               612652 non-null float64
2_total_fee               612652 non-null float64
3_total_fee               612652 non-null float64
4_total_fee               612652 non-null float64
month_traffic             612652 non-null float64
many_over_bill            612652 non-null int64
contract_type             612652 non-null int64
contract_time             612652 non-null int64
is_promise_low_consume    612652 non-null int64
net_service               612652 non-null int64
pay_times                 612652 non-null int64
pay_num                   612652 non-null float64
last_month_traffic        612652 non-null float64
local_trafffic_month      612652 non-null float64
local_caller_time         612652 non-null float64
service

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262565 entries, 0 to 262564
Data columns (total 26 columns):
service_type              262565 non-null int64
is_mix_service            262565 non-null int64
online_time               262565 non-null int64
1_total_fee               262565 non-null float64
2_total_fee               262565 non-null float64
3_total_fee               262565 non-null float64
4_total_fee               262565 non-null float64
month_traffic             262565 non-null float64
many_over_bill            262565 non-null int64
contract_type             262565 non-null int64
contract_time             262565 non-null int64
is_promise_low_consume    262565 non-null int64
net_service               262565 non-null int64
pay_times                 262565 non-null int64
pay_num                   262565 non-null float64
last_month_traffic        262565 non-null float64
local_trafffic_month      262565 non-null float64
local_caller_time         262565 non-null float64
service

In [8]:
ntrain = train.shape[0]
ntest = test.shape[0]
ntrain,ntest

(612652, 262565)

In [9]:
#test['current_service'] = None
train_copy = train.copy()

train_copy.drop('current_service', axis = 1, inplace=True)

train_test = pd.concat((train_copy, test)).reset_index(drop=True)

train_test.drop('user_id', axis = 1, inplace=True)

In [10]:
label = train.pop('current_service')
le = LabelEncoder()
train_y = le.fit_transform(label)

In [11]:
train_y

array([10,  6,  7, ..., 13,  5,  2], dtype=int64)

In [12]:
train_test['service_type'] = train_test['service_type'].astype(str)
train_test['is_mix_service'] = train_test['is_mix_service'].astype(str)
train_test['many_over_bill'] = train_test['many_over_bill'].astype(str)
train_test['contract_type'] = train_test['contract_type'].astype(str)
train_test['is_promise_low_consume'] = train_test['is_promise_low_consume'].astype(str)
train_test['net_service'] = train_test['net_service'].astype(str)
train_test['complaint_level'] = train_test['complaint_level'].astype(str)
train_test['gender'] = train_test['gender'].astype(str)

train_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875217 entries, 0 to 875216
Data columns (total 25 columns):
service_type              875217 non-null object
is_mix_service            875217 non-null object
online_time               875217 non-null int64
1_total_fee               875217 non-null float64
2_total_fee               875217 non-null float64
3_total_fee               875217 non-null float64
4_total_fee               875217 non-null float64
month_traffic             875217 non-null float64
many_over_bill            875217 non-null object
contract_type             875217 non-null object
contract_time             875217 non-null int64
is_promise_low_consume    875217 non-null object
net_service               875217 non-null object
pay_times                 875217 non-null int64
pay_num                   875217 non-null float64
last_month_traffic        875217 non-null float64
local_trafffic_month      875217 non-null float64
local_caller_time         875217 non-null float64
s

In [13]:
train_test = pd.get_dummies(train_test)

In [14]:
train_X = train_test[:ntrain]
test_X = train_test[ntrain:]

In [15]:
# 线性模型需要用标准化的数据建模

from sklearn.preprocessing import RobustScaler
ss2 = RobustScaler()
ss2.fit(train_X)
train_X_sd = ss2.transform(train_X)
test_X_sd = ss2.transform(test_X)

### 模型

In [16]:
def model_cv(model, x, y):
    score = cross_val_score(model, x, y,cv=5)
    return score

In [17]:
models = [LogisticRegression()]

# , Ridge(), Lasso(), RandomForestClassifier(), SVC(), GradientBoostingClassifier(), XGBClassifier()

In [111]:
names = ['LR']

#, 'Ridge', 'Lasso', 'RF', 'SVC', 'GBC', 'XGB'
for name, model in zip(names, models):
    Score = model_cv(model, train_X_sd,train_y)
    print("{}: {:.6f}, {:.4f}".format(name,Score.mean(),Score.std()))

LR: 0.449605, 0.1268


In [112]:
models = [RandomForestClassifier()]
names = ['rf']

#, 'Ridge', 'Lasso', 'RF', 'SVC', 'GBC', 'XGB'
for name, model in zip(names, models):
    Score = model_cv(model, train_X_sd,train_y)
    print("{}: {:.6f}, {:.4f}".format(name,Score.mean(),Score.std()))

rf: 0.913125, 0.0004


In [115]:
rf_model = RandomForestClassifier()
rf_model.fit(train_X, train_y)

pred = rf_model.predict(test_X)
pred = le.inverse_transform(pred)
test['predict'] = pred
test[['user_id', 'predict']].to_csv('./result/rf.csv', index=False)

F:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
sv_model = LinearSVC()
sv_model.fit(train_X, train_y)

pred = rf_model.predict(test_X)
pred = le.inverse_transform(pred)
test['predict'] = pred
test[['user_id', 'predict']].to_csv('./result/svc.csv', index=False)

In [17]:
GBDT_model = GradientBoostingClassifier()
GBDT_model.fit(train_X, train_y)

pred = GBDT_model.predict(test_X)
pred = le.inverse_transform(pred)
test['predict'] = pred
test[['user_id', 'predict']].to_csv('./result/GBDT.csv', index=False)

F:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(
                bjective='multiclass',
                boosting_type='gbdt',
                num_leaves=35,
                max_depth=8,
                learning_rate=0.05,
                seed=2018,
                colsample_bytree=0.8,
                subsample=0.9,
                n_estimators=2000)
clf.fit(train_X, train_y)
pred = clf.predict(test_X)
pred = le.inverse_transform(pred)
test['predict'] = pred
test[['user_id', 'predict']].to_csv('./result/clf.csv', index=False)

F:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
F:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
clf = XGBClassifier(max_depth=12, learning_rate=0.05,
                            n_estimators=752, silent=True,
                            objective="multi:softmax",
                            nthread=4, gamma=0,
                            max_delta_step=0, subsample=1, colsample_bytree=0.9, colsample_bylevel=0.9,
                            reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
                            base_score=0.5, seed=2018, missing=None,num_class=15)
clf.fit(train_X, train_y)
pred = clf.predict(test_X)
pred = le.inverse_transform(pred)
test['predict'] = pred
test[['user_id', 'predict']].to_csv('./result/xgbst.csv', index=False)